In [1]:
from ephys_queries import select_spike_times
from ephys_queries import db_setup_core
from dotenv import load_dotenv
from pathlib import Path

In [2]:
from spiketimes.df.binning import binned_spiketrain_bins_provided
from spiketimes.df.statistics import auc_roc_test_by
from spiketimes.df.statistics import diffmeans_test_by
from spiketimes.df.baseline import zscore_standardise_by
from spiketimes.utils import p_adjust

In [38]:
import numpy as np
import pandas as pd
from sklearn.metrics import auc
from datetime import datetime

In [4]:
data_dir = Path(".").absolute().parent / "data"
fig_dir = Path(".").absolute().parent / "figs"

In [5]:
group_names = ["acute_citalopram", 
            "acute_saline", 
            "acute_cit", 
            "acute_sal"]
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .loc[lambda x: x["group_name"].isin(group_names)]
)

In [14]:
df_cit = (
    pd.read_csv(data_dir / "cit_effects.csv")
    .loc[:, ["neuron_id", "response_dm", "diff_of_means", "diff_of_means_pct"]]
    .rename(columns={"response_dm": "citalopram_response", 
                     "diff_of_means": "citalopram_effect_delta_spikerate",
                     "diff_of_means_pct": "citalopram_effect_delta_pctbase"})
)

In [94]:
block_name = "way"
t_before = 10 * 60
fs = 30000
load_dotenv()
engine, metadata = db_setup_core()

date_after = datetime(2019, 6, 10)

df_spikes = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
        t_before=t_before,
        align_to_block=True,
        sessions_after=date_after
            )
    .assign(spiketimes=lambda x: x["spike_time_samples"].divide(fs))
)

In [95]:
def response_dm(row, p_col="p_dm", diff_mean_col="diff_of_means"):
    if row[p_col] >= 0.05:
        return "Non Responder"
    elif row[diff_mean_col] < 0:
        return "Inhibited"
    else:
        return "Stimulated"
    
def percent_baseline(df):
    mean_baseline = df.loc[lambda x: x.time < 0].spike_counts.mean()
    return df.assign(pct_baseline=lambda x: x.spike_counts.divide(mean_baseline) * 100)

In [96]:
df_spikes = (
    df_spikes
    .groupby("neuron_id")
    .filter(lambda x: x.spiketimes.max() > 1200)
)

In [101]:
bins = np.arange(-600, 1800, 30)

df = (
    binned_spiketrain_bins_provided(
        df_spikes, 
        bins, 
        spiketimes_col="spiketimes",
                               
        spiketrain_col="neuron_id"
    )
    .rename(columns={"counts": "spike_counts", "bin": "time"})
    .merge(dfb[["neuron_id", "cluster", "group_name"]])
    .assign(treatment=lambda x: x["group_name"].map({
                        "acute_cit": "Citalopram",
                        "acute_citalopram": "Citalopram",
                        "acute_saline": "Saline",
                        "acute_sal": "Saline"}),
           condition=lambda x: x["time"].apply(lambda y: "Baseline" if y < 0 else "Challenge"),
           firing_rate=lambda x: x["spike_counts"].divide(60),
           cond=lambda x: x["condition"].map({"Baseline":0, "Challenge":1}),

        )
    .drop("group_name", axis=1)
    .groupby("neuron_id")
    .apply(lambda x: percent_baseline(x))
    .reset_index(drop=True)
)

In [103]:
res_way = (
    diffmeans_test_by(
        df, 
        n_boot=1000, 
        spiketrain_col="neuron_id", 
        spikecount_col="firing_rate",
        condition_col="cond"
    )
    .rename(columns={"diff_of_means": "way_effect_DOM", "p": "p_way"})
    .assign(
        p =          lambda x: p_adjust(x.p_way),
        way_effect = lambda x: x.apply(response_dm, diff_mean_col="way_effect_DOM", p_col="p_way", axis=1)
    )
    .merge(df_cit)
    .merge(dfb[["neuron_id", "cluster"]])
)

In [104]:
res_way.to_csv(data_dir / "way_effects.csv", index=False)

In [105]:
res_way

,neuron_id,way_effect_DOM,p_way,p,way_effect,citalopram_response,citalopram_effect_delta_spikerate,citalopram_effect_delta_pctbase,cluster
0,1621,-0.228870,0.000,0.000000,Inhibited,Non Responder,-0.705000,-28.932969,slow_irregular
1,1622,-0.761921,0.000,0.000000,Inhibited,Non Responder,0.345000,7.931034,slow_irregular
2,1636,-1.268234,0.000,0.000000,Inhibited,Non Responder,2.230000,64.951456,slow_irregular
3,1645,-0.434675,0.000,0.000000,Inhibited,Non Responder,0.921667,29.699248,slow_irregular
4,1658,-0.003065,0.667,0.761204,Non Responder,Inhibited,-3.588333,-58.505435,slow_regular
...,...,...,...,...,...,...,...,...,...
123,1999,-0.043658,0.672,0.761204,Non Responder,Non Responder,0.960000,30.868167,slow_regular
124,2000,-0.059280,0.075,0.104348,Non Responder,Non Responder,-0.050000,-4.702194,no_baseline
125,2001,0.013771,0.168,0.215040,Non Responder,Non Responder,0.038333,12.921348,slow_irregular
126,2002,-0.627895,0.000,0.000000,Inhibited,Non Responder,-0.486667,-0.619064,fast_firing
